## import


In [34]:
# 최신 노선 정보 불러오기
import pandas as pd
import requests
import json
import xmltodict
from urllib.request import urlopen #request package
from bs4 import BeautifulSoup #BeautifulSoup
import pandas as pd # pandas

servicekey = 'Z%2FDKASzEwlIASKLfw%2FQQNTrexprISYo9aeuVrpI3dL2RyUpavjBPY6oJ5EPc5A6dE0XCA2HEURSO%2BOmu8lfw7w%3D%3D'

## api불러올 때 필요한 Busid

In [42]:
#서울시노선기본정보에서 노선코드 불러오기

seoulbus_info = pd.read_csv(r"C:\Users\0811m\Desktop\DScover\메인프로젝트\서울시노선기본정보.csv")
seoulbus_info = seoulbus_info[~((seoulbus_info["ROUTE_TY"] == 1) | (seoulbus_info["ROUTE_TY"] == 2))]
seoulbus_info = seoulbus_info[~(seoulbus_info["ROUTE_NM"].str.contains("N") | seoulbus_info["ROUTE_NM"].str.contains("심야"))]
seoulbus_info

,ROUTE_ID,ROUTE_NM,ROUTE_ABRV,DSTNC,ROUTE_TY,SSTTN_NM,ESTTN_NM,CARALC,FIRCAR_TM,LSTCAR_TM,GROUP_NM
0,100100124,0017,0017,12.20,4,청암동,이촌동,12,0.0515,0.2330,보광교통
1,100100001,01A,01A,16.00,5,예장주차장,예장주차장,8,0.0630,0.2300,북부운수
2,106000004,01B,01B,16.00,5,예장주차장,예장주차장,17,0.0630,0.2300,북부운수
3,104000012,0411,0411,44.30,4,용산차고지,AT센터.양재꽃시장,14,0.0420,0.2230,대원여객
4,100100549,100,100,57.09,3,하계동,용산구청,10,0.0400,0.2230,한성여객
...,...,...,...,...,...,...,...,...,...,...,...
2617,115000011,서울02,서울02,12.00,13,김포시풍무동,김포공항역,10,0.0630,0.0820,"공항버스, 다모아자동차"
2618,111000020,서울03,서울03,37.00,13,파주시운정지구,홍대입구역,20,0.0620,0.0700,선진운수
2619,111000021,서울04,서울04,13.00,13,고양시원흥지구,가양역,15,0.0630,0.0715,선진운수
2620,109000002,서울05,서울05,21.00,13,양주시옥정신도시,도봉산역,15,0.0630,0.0715,서울교통네트웍


In [45]:
bus_ids = seoulbus_info["ROUTE_ID"].unique().tolist()
bus_ids

[100100124,
 100100001,
 106000004,
 104000012,
 100100549,
 100100006,
 100100129,
 100100130,
 100100007,
 100100131,
 100100008,
 100100009,
 100100010,
 100100011,
 100100012,
 100100014,
 100100016,
 100100015,
 104000009,
 100100132,
 100100133,
 100100134,
 100100566,
 107000001,
 100100137,
 100100138,
 100100425,
 100100139,
 100100142,
 100100143,
 100100144,
 100100145,
 100100146,
 100100147,
 100100148,
 100100149,
 100100150,
 100100151,
 100100152,
 100100153,
 100100154,
 100100155,
 100100156,
 100100157,
 100100158,
 100100159,
 100100164,
 100100165,
 100100170,
 100100171,
 100100172,
 108000002,
 100100017,
 100100583,
 100100175,
 100100177,
 100100178,
 100100179,
 100100181,
 100100183,
 100100184,
 100100018,
 100100019,
 100100020,
 100100021,
 100100022,
 100100023,
 100100024,
 100100025,
 100100026,
 100100027,
 100100029,
 100100030,
 100100031,
 100100033,
 100100034,
 100100032,
 100100036,
 100100185,
 100100037,
 110000002,
 100100038,
 100100186,
 100

In [75]:
    # api로부터 데이터 요청
url = f'http://ws.bus.go.kr/api/rest/busRouteInfo/getRouteInfo?ServiceKey={servicekey}&busRouteId=100100134'
result = urlopen(url) #url 오픈
bus = BeautifulSoup(result,'lxml-xml') #url을 xml 방식으로 취합
te = bus.find_all("item") # item 항목만 추출하여 te로 저장
print(bus)

<?xml version="1.0" encoding="utf-8"?>
<ServiceResult><comMsgHeader/><msgHeader><headerCd>7</headerCd><headerMsg>Key인증실패: LIMITED NUMBER OF SERVICE REQUESTS EXCEEDS ERROR.[인증모듈 에러코드(22)]</headerMsg><itemCount>0</itemCount></msgHeader><msgBody/></ServiceResult>


# getRouteInfo

In [70]:
#getRouteInfo api 불러오기

data = []

for bus in bus_ids:
    bus_id = bus
    # api로부터 데이터 요청
    url = f'http://ws.bus.go.kr/api/rest/busRouteInfo/getRouteInfo?ServiceKey={servicekey}&busRouteId={bus_id}'

    result = urlopen(url) #url 오픈
    bus = BeautifulSoup(result,'lxml-xml') #url을 xml 방식으로 취합
    te = bus.find_all("itemList") # item 항목만 추출하여 te로 저장


    for item in te:
        노선ID = item.find("busRouteId").text if item.find("busRouteId") else None
        노선약칭 = item.find("busRouteAbrv").text if item.find("busRouteAbrv") else None
        노선명 = item.find("busRouteNm").text if item.find("busRouteNm") else None
        노선길이 = item.find("length").text if item.find("length") else None
        노선유형 = item.find("routeType").text if item.find("routeType") else None
        기점 = item.find("stStationNm").text if item.find("stStationNm") else None
        종점 = item.find("edStationNm").text if item.find("edStationNm") else None
        배차간격 = item.find("term").text if item.find("term") else None

        total = [노선ID, 노선약칭, 노선명, 노선길이, 노선유형, 기점, 종점, 배차간격]  # 추출된 데이터를 리스트의 형태로 취합
        data.append(total)  # total을 data로 누적
 
columns = ['노선ID', '노선약칭', '노선명', '노선길이', '노선유형', '기점', '종점', '배차간격']
df = pd.DataFrame(data, columns=columns)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


KeyboardInterrupt: 

In [69]:
df

,노선ID,노선약칭,노선명,노선길이,노선유형,기점,종점,배차간격


In [ ]:
df.to_csv(r"C:\Users\0811m\Desktop\DScover\메인프로젝트\getRoutePath.csv")

## getRoutePath

In [79]:
#getRoutePath api 불러오기

data = []

for bus in bus_ids:
    bus_id = bus
    # api로부터 데이터 요청
    url = f'http://ws.bus.go.kr/api/rest/busRouteInfo/getRoutePath?ServiceKey={servicekey}&busRouteId={bus_id}'

    result = urlopen(url) #url 오픈
    bus = BeautifulSoup(result,'lxml-xml') #url을 xml 방식으로 취합
    te = bus.find_all("itemList") # item 항목만 추출하여 te로 저장
    
    for item in te:
        순번 = item.find("no").text
        GPS좌표X = item.find("gpsX").text
        GPS좌표Y = item.find("gpsY").text
        GRS좌표X = item.find("posX").text
        GRS좌표Y = item.find("posY").text

        total = [bus_id, 순번,GPS좌표X,GPS좌표Y,GRS좌표X,GRS좌표Y ]  # 추출된 데이터를 리스트의 형태로 취합
        data.append(total)  # total을 data로 누적
 
columns = ['노선명', '순번', 'GPS좌표X', 'GPS좌표Y','GRS좌표X','GRS좌표Y' ]
df = pd.DataFrame(data, columns=columns)

MemoryError: 

In [78]:
df

,노선명,순번,GPS좌표X,GPS좌표Y,GRS좌표X,GRS좌표Y
0,"[[[], [<headerCd>0</headerCd>, <headerMsg>정상적으...",1,126.9484156209,37.5346252374,195441.32069447223,448353.8716791235
1,"[[[], [<headerCd>0</headerCd>, <headerMsg>정상적으...",2,126.9483834344,37.5346252374,195438.4762686148,448353.8732398064
2,"[[[], [<headerCd>0</headerCd>, <headerMsg>정상적으...",3,126.9483405191,37.5346294913,195434.6839630708,448354.34740228206
3,"[[[], [<headerCd>0</headerCd>, <headerMsg>정상적으...",4,126.9482600528,37.5346167296,195427.5721152252,448352.9350713417
4,"[[[], [<headerCd>0</headerCd>, <headerMsg>정상적으...",5,126.9480937558,37.5345912062,195412.8743425409,448350.1106893616
...,...,...,...,...,...,...
442702,"[[[], [<headerCd>0</headerCd>, <headerMsg>정상적으...",530,126.887883,37.515134,190089.27665745278,446195.47414812725
442703,"[[[], [<headerCd>0</headerCd>, <headerMsg>정상적으...",531,126.887841,37.515152,190085.56639466505,446197.47613210836
442704,"[[[], [<headerCd>0</headerCd>, <headerMsg>정상적으...",532,126.88611,37.515904,189932.6534778066,446281.11352901254
442705,"[[[], [<headerCd>0</headerCd>, <headerMsg>정상적으...",533,126.884169,37.516775,189761.19704623325,446377.98267594306


In [ ]:
df.to_csv(r"C:\Users\0811m\Desktop\DScover\메인프로젝트\getRoutePath.csv")